In [2]:
from IPython.core.display import display
from src.dataSetHandler import DatasetInfo, DatasetHandler
import pandas as pd
import numpy as np
from src.trainer import EPTrainer

dataset_info = DatasetInfo(
    name="ud_data",
    num_spans=2,
    frac=1
)

In [ ]:
from transformers import AutoTokenizer

model_name = 'xlnet-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
dataset_handler = DatasetHandler(
    root_dir_path="/content/edge-probing-datasets/data/",
    dataset_info=dataset_info,
    tokenizer=tokenizer
)

In [ ]:
# Check
rnd_idx = np.random.randint(100)
part = "train"

display(pd.DataFrame(dataset_handler.tokenized_dataset[part][0:3]))
print(dataset_handler.tokenized_dataset)
print("Original Spans:", dataset_handler.dataset_dict[part][rnd_idx])
print("Tokenized Spans:", dataset_handler.tokenized_dataset[part][rnd_idx])
test_tokens = tokenizer.convert_ids_to_tokens(dataset_handler.tokenized_dataset[part][rnd_idx]["input_ids"])
print(test_tokens)

s10, s11 = dataset_handler.tokenized_dataset[part][rnd_idx]["span1"][0], dataset_handler.tokenized_dataset[part][rnd_idx]["span1"][-1]
s20, s21 = dataset_handler.tokenized_dataset[part][rnd_idx]["span2"][0], dataset_handler.tokenized_dataset[part][rnd_idx]["span2"][-1]
print("span1:", s10, s11, test_tokens[s10:s11])
print("span2:", s20, s21, test_tokens[s20:s21])
print("label:", dataset_handler.tokenized_dataset[part][rnd_idx]["label"])

pd.DataFrame(dataset_handler.tokenized_dataset[part]["label"], columns=['label'])["label"].value_counts().plot(kind='barh', color="green", figsize=(10, 9));

In [ ]:
from transformers import XLNetModel

model = XLNetModel.from_pretrained(model_name, output_hidden_states=True)

In [ ]:
ep_trainer = EPTrainer(
    pretrained_model=model.to('cuda'),
    dataset_handler=dataset_handler,
    batch_size = 64
)

In [ ]:
%%time
ep_trainer.train(epochs=3)